In [2]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("自己紹介してください")
print(ai_message)



私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？


In [4]:
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは"),
]

for chunk in model.stream(messages):
  print(chunk.content, end="",flush=True)

こんにちは！どのようにお手伝いできますか？

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下のレシピを考えてください。

料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value.text)

以下のレシピを考えてください。

料理名: カレー


In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]


In [8]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [9]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
  ingredients: list[str] = Field(description= "ingredients of the dish")
  steps: list[str] = Field(description="Steps ti make the dish")


In [10]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [11]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "Steps ti make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [13]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: human ===")
print(prompt_value.messages[1].content)

=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "Steps ti make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: human ===
カレー


In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、さらに炒める。",
    "水を加え、沸騰したらアクを取り、弱火で20分煮る。",
    "カレールーを加え、溶かしながらさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


### StrOutputParser

In [15]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです")
output = output_parser.invoke(ai_message)
print(output)

こんにちは。私はAIアシスタントです


In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","ユーザーが入力した料理のレシピを教えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

chain = prompt | model

ai_message = chain.invoke({"dish":"カレー"})
print(ai_message)

content='カレーの基本的なレシピをご紹介します。以下は、一般的なチキンカレーの作り方です。\n\n### 材料\n- 鶏肉（もも肉または胸肉）: 500g\n- 玉ねぎ: 2個\n- にんにく: 2片\n- 生姜: 1片\n- トマト: 1個（またはトマト缶）\n- カレーパウダー: 大さじ2\n- 塩: 適量\n- 胡椒: 適量\n- サラダ油: 大さじ2\n- 水: 400ml\n- お好みで野菜（じゃがいも、にんじん、ピーマンなど）\n\n### 作り方\n1. **下ごしらえ**:\n   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。\n   - 玉ねぎは薄切り、にんにくと生姜はみじん切りにします。\n   - トマトはざく切りにします。\n\n2. **炒める**:\n   - 鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。\n   - にんにくと生姜を加え、香りが立つまでさらに炒めます。\n\n3. **鶏肉を加える**:\n   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。\n\n4. **スパイスを加える**:\n   - カレーパウダーを加え、全体に絡めるように炒めます。\n\n5. **トマトと水を加える**:\n   - トマトを加え、全体を混ぜたら、水を加えます。沸騰したら、アクを取り除きます。\n\n6. **煮込む**:\n   - 蓋をして、中火で約20分煮込みます。お好みで野菜を加える場合は、このタイミングで入れます。\n\n7. **味を調える**:\n   - 最後に塩で味を調整し、必要に応じてさらに煮込みます。\n\n8. **完成**:\n   - ご飯やナンと一緒に盛り付けて、お召し上がりください。\n\n### お好みで\n- ココナッツミルクを加えると、まろやかな味わいになります。\n- 辛さを調整したい場合は、チリパウダーや唐辛子を加えてください。\n\nぜひお試しください！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 608, 'prompt_tokens': 28, 'total_tokens': 636, 'completio

In [17]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish":"カレー"})
print(output)

カレーの基本的なレシピをご紹介します。これはシンプルなチキンカレーのレシピですが、具材やスパイスはお好みに応じてアレンジできます。

### 材料
- 鶏肉（もも肉または胸肉）: 500g
- 玉ねぎ: 2個
- にんにく: 2片
- 生姜: 1片
- トマト: 1個（またはトマト缶）
- カレーパウダー: 大さじ2
- 塩: 適量
- 胡椒: 適量
- サラダ油: 大さじ2
- 水: 500ml
- お好みで野菜（じゃがいも、にんじん、ピーマンなど）

### 作り方
1. **下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんにくと生姜はみじん切りにします。
   - トマトは角切りにします。

2. **炒める**:
   - 鍋にサラダ油を熱し、玉ねぎを加えて透明になるまで炒めます。
   - にんにくと生姜を加え、香りが立つまでさらに炒めます。

3. **鶏肉を加える**:
   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。

4. **スパイスを加える**:
   - カレーパウダーを加え、全体に絡めるように炒めます。

5. **トマトと水を加える**:
   - トマトを加え、全体を混ぜたら水を加えます。
   - 沸騰したら、アクを取り除き、弱火にして20〜30分煮込みます。

6. **野菜を加える（オプション）**:
   - お好みでじゃがいもやにんじんなどの野菜を加え、さらに煮込みます。

7. **味を調える**:
   - 最後に塩で味を調整し、必要に応じてカレーパウダーを追加します。

8. **完成**:
   - ご飯やナンと一緒に盛り付けて、お召し上がりください。

お好みで、ココナッツミルクやヨーグルトを加えると、まろやかな味わいになります。楽しんでください！


In [18]:
!pip install  GitPython==3.1.43

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [GitPython]/3 [GitPython]


In [19]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

36


In [20]:
# !pip install langchain-text-splitters==0.3.0

In [21]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1455, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000


96


In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [23]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.01968870498239994, -0.007562828715890646, 0.029617559164762497, -0.026554401963949203, 0.049940019845962524, 0.025012260302901268, -0.014977781102061272, 0.02129422128200531, 0.025730518624186516, -0.021463222801685333, 0.009273971430957317, -0.010615423321723938, -0.017417743802070618, -0.006010124925523996, -0.011428743600845337, 0.06316440552473068, 0.033335596323013306, -0.00034394499380141497, -0.04495446756482124, 0.02638540044426918, 0.0320047065615654, 0.03532136604189873, -0.03819439932703972, 0.021864602342247963, 0.01868525706231594, -0.018769757822155952, -0.020269649103283882, 0.032237086445093155, -0.00830221176147461, -0.10114755481481552, -0.009332065470516682, -0.0574183464050293, -0.034243982285261154, 0.04677123576402664, -0.023934874683618546, 0.034835487604141235, 0.023343367502093315, 0.013308888301253319, -0.00799589604139328, -0.033758100122213364, 0.00023254245752468705, -0.021009031683206558, 0.021072406321763992, 0.013171575032174587, 0.00648016110062

In [26]:
!pip install langchain-chroma==0.1.4 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 1.2.6 requires langchain-core<2.0.0,>=1.2.7, but you have langchain-core 0.3.83 which is incompatible.
langchain-text-splitters 1.1.0 requires langchain-core<2.0.0,>=1.2.0, but you have langchain-core 0.3.83 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.83 which is incompatible.
langgraph-prebuilt 1.0.6 requires langchain-core>=1.0.0, but you have langchain-core 0.3.83 which is incompatible.
langchain-classic 1.0.1 requires langchain-core<2.0.0,>=1.2.5, but you have langchain-core 0.3.83 which is incompatible.
langchain-openai 1.1.7 requires langchain-core<2.0.0,>=1.2.6, but you have langchain-core 0.3.83 which is incompatible.


In [27]:
from langchain_chroma.vectorstores import Chroma

db = Chroma.from_documents(docs, embedding=embeddings)

onnxruntime cpuid_info warning: Unknown CPU vendor. cpuinfo_vendor value: 0
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
